<a href="https://colab.research.google.com/github/MustiCankan/SpiriteAI/blob/main/GenerateImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!pip install -U diffusers transformers accelerate torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [9]:
import gradio as gr
from diffusers import DiffusionPipeline, FluxPipeline
import torch
import time

pipe = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

flux_pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)


def generate_image_with_progress(prompt, step_size, guidance_scale):
    step_size = int(step_size)
    guidance_scale = float(guidance_scale)

    # Create a Progress object
    progress = gr.Progress(track_tqdm=True)

    # Show simulated progress for the specified number of steps
    for i in progress.tqdm(range(step_size), desc="Generating steps"):
        time.sleep(0.05)  # Simulate work for each step

    # After simulating steps, run the actual pipeline
    image = pipe(
        prompt=prompt,
        num_inference_steps=step_size,
        guidance_scale=guidance_scale
    ).images[0]

    return image

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Group():
            gr.Markdown("### Spirit AI Model Extractor (SAME) v1.1.0", elem_id="name_l")
            logo = gr.Image(value="/content/spırıt.png", show_label=False, elem_id="logo-image")
        with gr.Column():
            prompt = gr.Textbox(label="Prompt")
            # Assign dropdowns to variables so we can access their values.
            pretrain_dropdown = gr.Dropdown(
                ["Flux", "Playground"],
                label="PretrainModels",
                info="Select the pre-trained model"
            )
            lora_dropdown = gr.Dropdown(
                ["Lora 1", "Lora 2", "Lora 3"],
                label="LoRAModules",
                info="Select one of LoRa Modules"
            )
            with gr.Accordion("Advanced options", open=False):
                step_size = gr.Number(label="Step Size", value=50)
                guidance_scale = gr.Number(label="Guidance Scale", value=3)
                negative_prompt = gr.Textbox(
                    label="Negative prompt",
                    max_lines=1,
                    placeholder="Enter a negative prompt",
                )


            generate_image_btn = gr.Button(value="Generate Image")
            examples = gr.Examples(
                examples=["Blonde hair girl", "Bald hair girl"],
                inputs=[prompt]
            )

        with gr.Column():
            output_image = gr.Image(label="Generated Image")

        # Custom CSS for resizing the logo
        demo.css = """
        #logo-image img {
            width: 200px;
            height: auto;
            display: block;
        }

        #name_l img {
            max-width: 100%;
            height: auto;
            display: block;
            margin: 5 auto;
            padding: 8;
        }
        """

    generate_image_btn.click(
        fn=generate_image_with_progress,
        inputs=[prompt, step_size, guidance_scale],
        outputs=output_image
    )

demo.launch(share=True, debug=True)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f59c557e17fba4afc8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f59c557e17fba4afc8.gradio.live


In [ ]:
import gradio as gr
from diffusers import DiffusionPipeline
import torch
import time

pipe = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic",
    torch_dtype=torch.float16,
    variant="fp16"
).to("cuda")

def generate_image_with_epochs(prompt, step_size, guidance_scale, negative_prompt=None):
    step_size = int(step_size)
    guidance_scale = float(guidance_scale)
    negative_prompt = negative_prompt if negative_prompt else ""

    # We'll store each "epoch" image here
    images = []

    # Create a Gradio progress object (no context manager)
    progress = gr.Progress(track_tqdm=True)

    # We iterate from 1 to step_size. On each iteration, we call
    # the pipeline with num_inference_steps=current_steps.
    for current_step in progress.tqdm(range(1, step_size + 1), desc="Epoch"):
        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=current_step,
            guidance_scale=guidance_scale
        ).images[0]
        images.append(result)

    return images

def clear_gallery():
    """
    Return an empty list to reset the gallery.
    """
    return []

with gr.Blocks() as demo:
    gr.Markdown("## Generate Images at Each Step (Simulated)")

    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="blurry")
            step_size = gr.Number(label="Max Steps (Epochs)", value=10)
            guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            generate_btn = gr.Button("Generate")
            clear_btn = gr.Button("Clear Gallery")

        with gr.Column():
            # We'll show the output as a Gallery to see multiple images
            gallery = gr.Gallery(label="Images at each step")

    # Clicking 'Generate' runs the pipeline function
    generate_btn.click(
        fn=generate_image_with_epochs,
        inputs=[prompt, step_size, guidance_scale, negative_prompt],
        outputs=gallery
    )

    # Clicking 'Clear Gallery' empties the gallery
    gallery= clear_btn.click(
        fn=clear_gallery,
        inputs=None,
        outputs=gallery
    )

demo.launch(debug=True, share=True)

Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  4.29steps/s]


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://586531916c353d2e55.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/1 [00:00<?, ?steps/s]

  0%|          | 0/2 [00:00<?, ?steps/s]

  0%|          | 0/3 [00:00<?, ?steps/s]

...

  0%|          | 0/17 [00:00<?, ?steps/s]

  0%|          | 0/18 [00:00<?, ?steps/s]

  0%|          | 0/19 [00:00<?, ?steps/s]

  0%|          | 0/20 [00:00<?, ?steps/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://586531916c353d2e55.gradio.live


In [10]:
import gradio as gr
from diffusers import DiffusionPipeline, FluxPipeline
import torch
import time

# Load your pipelines
pipe = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

flux_pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-schnell",
    torch_dtype=torch.bfloat16
)

def generate_image_with_progress(prompt, pretrain_model, lora_module, step_size, guidance_scale, negative_prompt):
    step_size = int(step_size)
    guidance_scale = float(guidance_scale)

    # Simulated progress bar
    progress = gr.Progress(track_tqdm=True)
    for i in progress.tqdm(range(step_size), desc="Generating steps"):
        time.sleep(0.05)  # Simulate work for each step

    # Use the selected pipeline based on the dropdown choice
    if pretrain_model == "Flux":
        # Optionally, you can add logic to handle lora_module if needed.
        result = flux_pipe(
            prompt=prompt,
            num_inference_steps=step_size,
            guidance_scale=guidance_scale
        )
    else:  # "Playground" selected
        result = pipe(
            prompt=prompt,
            num_inference_steps=step_size,
            guidance_scale=guidance_scale
        )

    image = result.images[0]
    return image

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Group():
            gr.Markdown("### Spirit AI Model Extractor (SAME) v1.1.0", elem_id="name_l")
            logo = gr.Image(value="/content/spırıt.png", show_label=False, elem_id="logo-image")
        with gr.Column():
            prompt = gr.Textbox(label="Prompt")
            # Assign dropdowns to variables so we can access their values.
            pretrain_dropdown = gr.Dropdown(
                ["Flux", "Playground"],
                label="PretrainModels",
                info="Select the pre-trained model"
            )
            lora_dropdown = gr.Dropdown(
                ["Lora 1", "Lora 2", "Lora 3"],
                label="LoRAModules",
                info="Select one of LoRa Modules"
            )
            with gr.Accordion("Advanced options", open=False):
                step_size = gr.Number(label="Step Size", value=50)
                guidance_scale = gr.Number(label="Guidance Scale", value=3)
                negative_prompt = gr.Textbox(
                    label="Negative prompt",
                    max_lines=1,
                    placeholder="Enter a negative prompt",
                )

            generate_image_btn = gr.Button(value="Generate Image")
            examples = gr.Examples(
                examples=["Blonde hair girl", "Bald hair girl"],
                inputs=[prompt]
            )
        with gr.Column():
            output_image = gr.Image(label="Generated Image")

        # Custom CSS for resizing the logo
        demo.css = """
        #logo-image img {
            width: 200px;
            height: auto;
            display: block;
        }

        #name_l img {
            max-width: 100%;
            height: auto;
            display: block;
            margin: 5 auto;
            padding: 8;
        }
        """

    # Update the inputs list to include the dropdown values
    generate_image_btn.click(
        fn=generate_image_with_progress,
        inputs=[prompt, pretrain_dropdown, lora_dropdown, step_size, guidance_scale, negative_prompt],
        outputs=output_image
    )

demo.launch(share=True, debug=True)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://63f8e91f9528a8be08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?steps/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://63f8e91f9528a8be08.gradio.live
